In [1]:
from  IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

import random
import time
import sys

In [3]:
def gpu_check(seed_val = 1):
    print('The Seed is set to {}'.format(seed_val))
    if torch.cuda.is_available():
        print('Model will Run on CUDA.')
        print ("Type 'watch nvidia-smi' to monitor GPU\n")
        torch.cuda.manual_seed(seed_val)
        device = 'cuda'
    else:
        torch.manual_seed(seed_val)
        print ('Running in CPU')
        device = 'cpu'
    cuda = torch.cuda.is_available()
    return cuda,seed_val,device

In [4]:
cuda,SEED,device = gpu_check(seed_val=1234)

The Seed is set to 1234
Model will Run on CUDA.
Type 'watch nvidia-smi' to monitor GPU



In [5]:
torch.backends.cudnn.deterministic = True

In [6]:
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [7]:
type(TEXT)
type(LABEL)

torchtext.legacy.data.field.Field

torchtext.legacy.data.field.LabelField

In [8]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL,root = '/home/jd/Desktop/DATASET/IMDB_data')

In [9]:
line = ''
filenumber = 10

for i in train_data.examples[filenumber].text:
    line += i + ' '
line
train_data.examples[filenumber].label

"I just saw this film @ TIFF ( Toronto International Film Festival ) . Fans of Hal Hartley will not be disappointed ! ! And if you are not familiar with this director 's oeuvre ... does n't matter . This film can definitely stand all on its own . I have to go the second screening ... it was amazing I need to see it again -- and fast ! ! < br /><br />This film is very funny . It 's dialogue is very smart , and the performance of Parker Posey is outstanding as she stars in the title role of Fay Grim . Fay Grim is the latest feature revisiting the world and characters introduced in the film Henry Fool ( 2000 ) . Visually , the most salient stylistic feature employs the habitual use of the canted ( or dutch ) angle , which can be often seen in past Hartley works appearing in various shorts , available in the Possible Films : short works by Hal Hartley 1994 - 2004 collection , and in The Girl from Monday ( 2005).<br /><br />I viewed this film most aptly on Sept 11th . Textually , Fay Grim '

'pos'

In [10]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [11]:

print(vars(train_data.examples[0]))

{'text': ['Centered', 'in', 'the', 'downtown', 'and', 'out', 'skirts', 'of', 'Detroit', ',', 'this', 'comedy', 'I', 'found', 'to', 'be', 'a', 'terrific', 'new', 'comedic', 'duo', '.', "'", 'Noriyuki', 'Pat', 'Morita', "'", 'is', 'a', 'very', 'funny', 'man', ',', 'who', 'happens', 'to', 'be', 'a', 'cop', 'from', 'Japan', 'on', 'the', 'trail', 'of', 'an', 'industrial', 'secrets', 'thief', ',', 'who', 'has', 'stolen', 'a', "'", 'proto', 'type', "'", 'turbo', 'super', 'charger', ',', 'reluctantly', 'he', 'goes', 'to', 'the', 'United', 'States', 'to', 'follow', 'the', 'thief', ',', 'after', 'being', 'ordered', 'by', 'his', 'commander', '.', 'Pat', "'s", 'character', 'collides', 'with', "'", 'Jay', 'Leno', "'s", "'", 'character', ',', 'a', 'fast', 'talking', "'", 'but', 'down', 'to', 'business', '-', 'player', 'type', 'Detroit', 'cop', '.', 'When', 'they', 'cross', 'paths', 'though', ',', 'the', 'honorable', "'", 'Ways', "'", 'of', 'Japan', 'meet', 'the', 'all', '-', 'out', 'old', 'school', 

In [12]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [13]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [14]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE) #Construct the Vocab object for this field from one or more datasets.
LABEL.build_vocab(train_data)

In [15]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [16]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202895), (',', 193215), ('.', 166188), ('a', 109830), ('and', 109762), ('of', 101071), ('to', 93999), ('is', 76505), ('in', 61400), ('I', 54509), ('it', 53489), ('that', 49345), ('"', 44905), ("'s", 43571), ('this', 42386), ('-', 37875), ('/><br', 35672), ('was', 34875), ('as', 30563), ('with', 30194)]


In [17]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [18]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [19]:
BATCH_SIZE = 32
#Create Iterator objects for multiple splits of a dataset.
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [20]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [21]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [23]:
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [24]:
criterion = nn.BCEWithLogitsLoss()

In [25]:
model = model.to(device)
criterion = criterion.to(device)

In [26]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [27]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 35s
	Train Loss: 0.694 | Train Acc: 49.64%
	 Val. Loss: 0.696 |  Val. Acc: 51.29%
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 0.693 | Train Acc: 50.34%
	 Val. Loss: 0.695 |  Val. Acc: 49.96%
Epoch: 03 | Epoch Time: 0m 32s
	Train Loss: 0.693 | Train Acc: 50.12%
	 Val. Loss: 0.695 |  Val. Acc: 49.95%
Epoch: 04 | Epoch Time: 0m 32s
	Train Loss: 0.693 | Train Acc: 50.25%
	 Val. Loss: 0.695 |  Val. Acc: 49.59%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.693 | Train Acc: 49.96%
	 Val. Loss: 0.695 |  Val. Acc: 49.67%


In [31]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

In [32]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.698 | Test Acc: 46.74%
